In [29]:
%matplotlib inline
import matplotlib.pyplot as plt      

import torch
from torch import nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.optim import Adam

import hyperparams as hp
from datasets import LJSpeechDataset, collate_fn
from visualize import show_spectrogram
from audio import wav_to_spectrogram
from text import text_to_sequence, sequence_to_text
from models import MelSpectrogramNet

from torch.utils.data import Subset

ImportError: cannot import name 'Subset'

In [2]:
x = Variable(torch.rand(1, 1, 9))

In [3]:

class CausalConv1d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, dilation, **kwargs):
        super().__init__()
        pad = (kernel_size - 1) * dilation
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size, padding=pad, dilation=dilation, **kwargs)

    def forward(self, x):
        x = self.conv1(x)
        x = x[:, :, :-self.conv1.padding[0]]  # remove trailing padding
        return x

In [4]:
conv = CausalConv1d(1, 1, kernel_size=3, dilation=2, bias=False)
x = torch.autograd.Variable(torch.randn(1, 1, 9))
x[:, :, :3].data.zero_() # make the first three elements zero
print(x)
print(conv(x))  # first three outputs should be zero

Variable containing:
(0 ,.,.) = 
  0.0000  0.0000  0.0000 -1.9275 -0.0328 -0.1590 -0.7235  1.5539  0.8788
[torch.FloatTensor of size 1x1x9]

Variable containing:
(0 ,.,.) = 
  0.0000  0.0000  0.0000  0.9078  0.0155 -0.0838  0.3380 -1.5032 -0.4864
[torch.FloatTensor of size 1x1x9]



In [5]:
t = conv(x)

In [20]:
first_3 = t[:, :, :3].data

In [25]:
(first_3 == 0).all()

True

In [28]:
(torch.Tensor([1, 1, 1]) == 0)


 0
 0
 0
[torch.ByteTensor of size 3]